In [1]:
import sys
import tensorflow as tf
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

2024-09-04 14:02:00.271594: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-04 14:02:00.271880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-04 14:02:00.588195: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-04 14:02:01.284677: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-04 14:02:22.881661: W tensorflow/compiler/tf2

In [2]:
print(tf.config.list_physical_devices('GPU'))

[]


2024-09-04 14:02:34.063662: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-09-04 14:02:34.063717: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: renan-Latitude-3450
2024-09-04 14:02:34.063739: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: renan-Latitude-3450
2024-09-04 14:02:34.063866: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.183.1
2024-09-04 14:02:34.063910: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.183.1
2024-09-04 14:02:34.063921: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.183.1


# Vanishing and Exploding Gradients Problem

### Xavier and He Initialization

Xavier or He is chosen by the activation that I want to use

In [3]:
# Xavier initialization
dense = tf.keras.layers.Dense(50, activation="relu",
                             kernel_initializer="he_normal")

In [4]:
# He initialization
he_avg_init = tf.keras.initializers.VarianceScaling(scale=2., mode="fan_avg", 
                                                    distribution="uniform")
dense = tf.keras.layers.Dense(50, activation="sigmoid",
                             kernel_initializer=he_avg_init)

### Nonsaturating Activation Functions

In [5]:
# Leaky ReLU

leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)
dense = tf.keras.layers.Dense(30, activation=leaky_relu,
                             kernel_initializer="he_normal")

Create a `LeakyReLU` layer and add it to the model right after the layer you want to apply it to.

In [6]:
model = tf.keras.models.Sequential([
    # more layers
    tf.keras.layers.Dense(50, kernel_initializer="he_normal"),  # no activation
    tf.keras.layers.LeakyReLU(alpha=0.2),  # activation as a separete layer
    # more layers
])

In [7]:
# ELU
dense = tf.keras.layers.Dense(50, activation="elu",
                             kernel_initializer="he_normal")

In [8]:
# SELU
dense = tf.keras.layers.Dense(50, activation="selu",
                             kernel_initializer="lecun_normal")

In [9]:
# Creating a NN using SELU
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[28, 28]))
for layer in range(100):
    model.add(tf.keras.layers.Dense(100, activation="selu",
                                    kernel_initializer="lecun_normal"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              metrics=["accuracy"])

In [10]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255, X_valid / 255, X_test / 255

In [11]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [12]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [13]:
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

2024-09-04 14:02:39.541153: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 172480000 exceeds 10% of free system memory.


Epoch 1/5
1719/1719 [==============================] - 32s 14ms/step - loss: 1.3171 - accuracy: 0.4902 - val_loss: 1.0487 - val_accuracy: 0.5708
Epoch 2/5
1719/1719 [==============================] - 26s 15ms/step - loss: 0.9085 - accuracy: 0.6607 - val_loss: 0.8036 - val_accuracy: 0.7054
Epoch 3/5
1719/1719 [==============================] - 26s 15ms/step - loss: 0.8494 - accuracy: 0.6875 - val_loss: 0.7313 - val_accuracy: 0.7398
Epoch 4/5
1719/1719 [==============================] - 28s 16ms/step - loss: 0.7233 - accuracy: 0.7368 - val_loss: 0.6354 - val_accuracy: 0.7750
Epoch 5/5
1719/1719 [==============================] - 24s 14ms/step - loss: 0.6577 - accuracy: 0.7596 - val_loss: 0.8431 - val_accuracy: 0.7186


### Batch Normalization

In [14]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation="relu",
                         kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu",
                         kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 batch_normalization (Batch  (None, 784)               3136      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 batch_normalization_1 (Bat  (None, 300)               1200      
 chNormalization)                                                
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Bat  (None, 100)               4

In [17]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd",
             metrics=["accuracy"])
model.fit(X_train, y_train, epochs=2, validation_data=(X_valid, y_valid))

Epoch 1/2


2024-09-04 14:04:57.754923: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 172480000 exceeds 10% of free system memory.


1719/1719 [==============================] - 8s 4ms/step - loss: 0.5489 - accuracy: 0.8102 - val_loss: 0.3990 - val_accuracy: 0.8540
Epoch 2/2
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4029 - accuracy: 0.8569 - val_loss: 0.3639 - val_accuracy: 0.8646


In [18]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

In [19]:
# BN before the activation function (sometimes works better)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False), # There is no need to use bias because BN does not need them
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [20]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd",
             metrics="accuracy")
model.fit(X_train, y_train, epochs=2, validation_data=(X_valid, y_valid))

Epoch 1/2


2024-09-04 14:05:13.830959: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 172480000 exceeds 10% of free system memory.


1719/1719 [==============================] - 10s 5ms/step - loss: 0.5644 - accuracy: 0.8059 - val_loss: 0.4189 - val_accuracy: 0.8514
Epoch 2/2
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4056 - accuracy: 0.8580 - val_loss: 0.3678 - val_accuracy: 0.8704


### Gradient Clipping

In [21]:
optimizer = tf.keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer)

In [22]:
optimizer = tf.keras.optimizers.SGD(clipnorm=1.0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer)

# Reusing Pretrained Layers

The book says that in the following situation is not good reuse the hidden layers of the trained model and the results are manipulated.
But I will do it anyway because later in the book it will show another example that might be useful.

In [23]:
import numpy as np

pos_class_id = class_names.index("Pullover")
neg_class_id = class_names.index("T-shirt/top")

def split_dataset(X, y):
    y_for_B = (y == pos_class_id) | (y == neg_class_id)
    y_A = y[~y_for_B]
    y_B = (y[y_for_B] == pos_class_id).astype(np.float32)
    old_class_ids = list(set(range(10)) - set([neg_class_id, pos_class_id]))
    for old_class_id, new_class_id in zip(old_class_ids, range(8)):
        y_A[y_A == old_class_id] = new_class_id  # reorder class ids for A
    return ((X[~y_for_B], y_A), (X[y_for_B], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [24]:
tf.random.set_seed(42)

model_A = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                         kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                         kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                         kernel_initializer="he_normal"),
    tf.keras.layers.Dense(8, activation="softmax")
])

model_A.compile(loss="sparse_categorical_crossentropy",
               optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
               metrics=["accuracy"])
history = model.fit(X_train_A, y_train_A, epochs=20,
                   validation_data=(X_valid_A, y_valid_A))
model_A.save("my_model_A")

Epoch 1/20


2024-09-04 14:05:32.734657: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 138018496 exceeds 10% of free system memory.


1376/1376 [==============================] - 7s 4ms/step - loss: 0.5800 - val_loss: 0.2731
Epoch 2/20
1376/1376 [==============================] - 6s 5ms/step - loss: 0.2719 - val_loss: 0.2418
Epoch 3/20
1376/1376 [==============================] - 5s 4ms/step - loss: 0.2406 - val_loss: 0.2303
Epoch 4/20
1376/1376 [==============================] - 5s 4ms/step - loss: 0.2178 - val_loss: 0.2193
Epoch 5/20
1376/1376 [==============================] - 5s 4ms/step - loss: 0.2038 - val_loss: 0.2142
Epoch 6/20
1376/1376 [==============================] - 5s 4ms/step - loss: 0.1935 - val_loss: 0.2092
Epoch 7/20
1376/1376 [==============================] - 5s 4ms/step - loss: 0.1827 - val_loss: 0.2038
Epoch 8/20
1376/1376 [==============================] - 5s 4ms/step - loss: 0.1739 - val_loss: 0.2095
Epoch 9/20
1376/1376 [==============================] - 5s 4ms/step - loss: 0.1679 - val_loss: 0.2022
Epoch 10/20
1376/1376 [==============================] - 5s 4ms/step - loss: 0.1596 - val_los

INFO:tensorflow:Assets written to: my_model_A/assets


In [25]:
tf.random.set_seed(42)
model_B = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                         kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                         kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                         kernel_initializer="he_normal"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_B.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
               metrics=["accuracy"])

history = model_B.fit(X_train_B, y_train_B, epochs=20,
                     validation_data=(X_valid_B, y_valid_B))
model_B.evaluate(X_test_B, y_test_B)

Epoch 1/20
7/7 [==============================] - 1s 40ms/step - loss: 0.7095 - accuracy: 0.4450 - val_loss: 0.6744 - val_accuracy: 0.4866
Epoch 2/20
7/7 [==============================] - 0s 13ms/step - loss: 0.6832 - accuracy: 0.4500 - val_loss: 0.6527 - val_accuracy: 0.4906
Epoch 3/20
7/7 [==============================] - 0s 13ms/step - loss: 0.6589 - accuracy: 0.4600 - val_loss: 0.6324 - val_accuracy: 0.5509
Epoch 4/20
7/7 [==============================] - 0s 13ms/step - loss: 0.6351 - accuracy: 0.5350 - val_loss: 0.6167 - val_accuracy: 0.6647
Epoch 5/20
7/7 [==============================] - 0s 13ms/step - loss: 0.6172 - accuracy: 0.6700 - val_loss: 0.6020 - val_accuracy: 0.7567
Epoch 6/20
7/7 [==============================] - 0s 13ms/step - loss: 0.5999 - accuracy: 0.7850 - val_loss: 0.5892 - val_accuracy: 0.8170
Epoch 7/20
7/7 [==============================] - 0s 13ms/step - loss: 0.5846 - accuracy: 0.8400 - val_loss: 0.5776 - val_accuracy: 0.8368
Epoch 8/20
7/7 [===========

[0.47289589047431946, 0.9079999923706055]

In [26]:
# reusing a pretrained model_A adding the last sigmoid layer
model_A = tf.keras.models.load_model("my_model_A")
model_B_on_A = tf.keras.Sequential(model_A.layers[:-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

tf.random.set_seed(42)

model_A_clone = tf.keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

model_B_on_A = tf.keras.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])


In [27]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 38ms/step - loss: 0.7500 - accuracy: 0.4450 - val_loss: 0.7430 - val_accuracy: 0.4837
Epoch 2/4
7/7 [==============================] - 0s 13ms/step - loss: 0.7448 - accuracy: 0.4400 - val_loss: 0.7386 - val_accuracy: 0.4827
Epoch 3/4
7/7 [==============================] - 0s 13ms/step - loss: 0.7394 - accuracy: 0.4450 - val_loss: 0.7337 - val_accuracy: 0.4758
Epoch 4/4
7/7 [==============================] - 0s 13ms/step - loss: 0.7332 - accuracy: 0.4550 - val_loss: 0.7302 - val_accuracy: 0.4758
Epoch 1/16
7/7 [==============================] - 1s 37ms/step - loss: 0.7212 - accuracy: 0.4600 - val_loss: 0.7113 - val_accuracy: 0.4857
Epoch 2/16
7/7 [==============================] - 0s 13ms/step - loss: 0.7006 - accuracy: 0.4800 - val_loss: 0.6945 - val_accuracy: 0.5232
Epoch 3/16
7/7 [==============================] - 0s 13ms/step - loss: 0.6817 - accuracy: 0.5750 - val_loss: 0.6786 - val_accuracy: 0.6073
Epoch 4/16
7/7 [===============

In [28]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.5530 - accuracy: 0.8900


[0.5529834032058716, 0.8899999856948853]

The results of the book is better in the "model_B_on_A" but they did it in a way to  get a better result.
So this is not unusual to get a worst result knowing that this is not a suitable way to use pretreined layers

# Faster Optimizers

In [29]:
def build_model(seed=42):
    tf.random.set_seed(seed)
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=[28, 28]),
        tf.keras.layers.Dense(100, activation="relu",
                             kernel_initializer="he_normal"),
        tf.keras.layers.Dense(100, activation="relu",
                             kernel_initializer="he_normal"),
        tf.keras.layers.Dense(100, activation="relu",
                             kernel_initializer="he_normal"),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

def build_and_train_model(optimizer):
    model = build_model()
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                 metrics=["accuracy"])
    return model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid,  y_valid))

In [30]:
optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

In [31]:
history_sgd = build_and_train_model(optimizer)

Epoch 1/10


2024-09-04 14:07:27.030488: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 172480000 exceeds 10% of free system memory.


1719/1719 [==============================] - 5s 2ms/step - loss: 0.6838 - accuracy: 0.7638 - val_loss: 0.4954 - val_accuracy: 0.8204
Epoch 2/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4619 - accuracy: 0.8366 - val_loss: 0.4429 - val_accuracy: 0.8426
Epoch 3/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4181 - accuracy: 0.8503 - val_loss: 0.4068 - val_accuracy: 0.8596
Epoch 4/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3919 - accuracy: 0.8610 - val_loss: 0.3868 - val_accuracy: 0.8618
Epoch 5/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3706 - accuracy: 0.8666 - val_loss: 0.3698 - val_accuracy: 0.8644
Epoch 6/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3576 - accuracy: 0.8715 - val_loss: 0.3643 - val_accuracy: 0.8686
Epoch 7/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3418 - accuracy: 0.8766 - val_loss: 0.3709 - val_accuracy: 0.8658
Epo

### Nesterov Accelerated Gradient

In [32]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9,
                                   nesterov=True)

In [33]:
history_nestorov = build_and_train_model(optimizer)

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6884 - accuracy: 0.7675 - val_loss: 0.4901 - val_accuracy: 0.8250
Epoch 2/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.4545 - accuracy: 0.8407 - val_loss: 0.4329 - val_accuracy: 0.8416
Epoch 3/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4121 - accuracy: 0.8554 - val_loss: 0.3973 - val_accuracy: 0.8572
Epoch 4/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3869 - accuracy: 0.8649 - val_loss: 0.3763 - val_accuracy: 0.8666
Epoch 5/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3666 - accuracy: 0.8699 - val_loss: 0.3650 - val_accuracy: 0.8688
Epoch 6/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3522 - accuracy: 0.8741 - val_loss: 0.3716 - val_accuracy: 0.8692
Epoch 7/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3375 - accuracy: 0.8784 - val_loss: 0.3672 - val_accuracy:

### AdaGrad

In [34]:
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.001)
history_adagrad = build_and_train_model(optimizer)

Epoch 1/10
1719/1719 [==============================] - 5s 2ms/step - loss: 0.9321 - accuracy: 0.7009 - val_loss: 0.6631 - val_accuracy: 0.7832
Epoch 2/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.6199 - accuracy: 0.7930 - val_loss: 0.5714 - val_accuracy: 0.8044
Epoch 3/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5570 - accuracy: 0.8112 - val_loss: 0.5301 - val_accuracy: 0.8156
Epoch 4/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5241 - accuracy: 0.8215 - val_loss: 0.5077 - val_accuracy: 0.8232
Epoch 5/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5027 - accuracy: 0.8275 - val_loss: 0.4902 - val_accuracy: 0.8272
Epoch 6/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4870 - accuracy: 0.8319 - val_loss: 0.4792 - val_accuracy: 0.8322
Epoch 7/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4750 - accuracy: 0.8368 - val_loss: 0.4681 - val_accuracy:

### RMSProp

In [35]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

In [36]:
history_rmsprop = build_and_train_model(optimizer)

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5128 - accuracy: 0.8154 - val_loss: 0.4392 - val_accuracy: 0.8368
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3920 - accuracy: 0.8585 - val_loss: 0.4615 - val_accuracy: 0.8338
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3661 - accuracy: 0.8697 - val_loss: 0.3902 - val_accuracy: 0.8626
Epoch 4/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3597 - accuracy: 0.8747 - val_loss: 0.3965 - val_accuracy: 0.8732
Epoch 5/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3526 - accuracy: 0.8771 - val_loss: 0.3873 - val_accuracy: 0.8564
Epoch 6/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3506 - accuracy: 0.8773 - val_loss: 0.4189 - val_accuracy: 0.8606
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3534 - accuracy: 0.8776 - val_loss: 0.4040 - val_accuracy:

### Adam Optimization

In [37]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [38]:
history_adam = build_and_train_model(optimizer)

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5003 - accuracy: 0.8203 - val_loss: 0.4092 - val_accuracy: 0.8416
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3747 - accuracy: 0.8634 - val_loss: 0.4039 - val_accuracy: 0.8448
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3406 - accuracy: 0.8750 - val_loss: 0.3502 - val_accuracy: 0.8762
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3163 - accuracy: 0.8833 - val_loss: 0.3429 - val_accuracy: 0.8720
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2978 - accuracy: 0.8891 - val_loss: 0.3298 - val_accuracy: 0.8746
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2826 - accuracy: 0.8943 - val_loss: 0.3550 - val_accuracy: 0.8732
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2709 - accuracy: 0.8971 - val_loss: 0.3586 - val_accuracy:

In [39]:
optimizer = tf.keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9,
                                       beta_2=0.999)
history_adamax = build_and_train_model(optimizer)

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5322 - accuracy: 0.8131 - val_loss: 0.4317 - val_accuracy: 0.8404
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3973 - accuracy: 0.8587 - val_loss: 0.3901 - val_accuracy: 0.8562
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3596 - accuracy: 0.8707 - val_loss: 0.3690 - val_accuracy: 0.8698
Epoch 4/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3345 - accuracy: 0.8798 - val_loss: 0.3465 - val_accuracy: 0.8744
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3143 - accuracy: 0.8845 - val_loss: 0.3271 - val_accuracy: 0.8756
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3000 - accuracy: 0.8896 - val_loss: 0.3343 - val_accuracy: 0.8782
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2866 - accuracy: 0.8941 - val_loss: 0.3311 - val_accuracy:

# Learning Rate Scheduling

### Power Scheduling

In [40]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    initial_learning_rate=0.01,
    decay_steps=10_000,
    decay_rate=1.0,
    staircase=False
)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

In [41]:
history_power_scheduling = build_and_train_model(optimizer)

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.6844 - accuracy: 0.7633 - val_loss: 0.4937 - val_accuracy: 0.8272
Epoch 2/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4729 - accuracy: 0.8345 - val_loss: 0.4442 - val_accuracy: 0.8374
Epoch 3/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4266 - accuracy: 0.8493 - val_loss: 0.4193 - val_accuracy: 0.8520
Epoch 4/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4029 - accuracy: 0.8574 - val_loss: 0.3974 - val_accuracy: 0.8596
Epoch 5/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3840 - accuracy: 0.8646 - val_loss: 0.3831 - val_accuracy: 0.8638
Epoch 6/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3703 - accuracy: 0.8693 - val_loss: 0.3890 - val_accuracy: 0.8630
Epoch 7/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3600 - accuracy: 0.8725 - val_loss: 0.3740 - val_accuracy:

### Exponential Scheduling

In [42]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=20_000,
    decay_rate=0.1,
    staircase=False
)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

In [43]:
history_exponential_scheduling = build_and_train_model(optimizer)

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.6774 - accuracy: 0.7687 - val_loss: 0.4909 - val_accuracy: 0.8304
Epoch 2/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4723 - accuracy: 0.8336 - val_loss: 0.4506 - val_accuracy: 0.8342
Epoch 3/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4284 - accuracy: 0.8476 - val_loss: 0.4152 - val_accuracy: 0.8522
Epoch 4/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4061 - accuracy: 0.8568 - val_loss: 0.3941 - val_accuracy: 0.8578
Epoch 5/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3890 - accuracy: 0.8625 - val_loss: 0.3888 - val_accuracy: 0.8594
Epoch 6/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3770 - accuracy: 0.8665 - val_loss: 0.3906 - val_accuracy: 0.8594
Epoch 7/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3686 - accuracy: 0.8688 - val_loss: 0.3753 - val_accuracy:

### Picewise Constant Scheduling

In [44]:
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[50_000, 80_000],
    values=[0.01, 0.005, 0.001]
)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

In [45]:
history_piecewise_scheduling = build_and_train_model(optimizer)

Epoch 1/10
1719/1719 [==============================] - 5s 2ms/step - loss: 0.6846 - accuracy: 0.7660 - val_loss: 0.4870 - val_accuracy: 0.8290
Epoch 2/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4673 - accuracy: 0.8348 - val_loss: 0.4629 - val_accuracy: 0.8282
Epoch 3/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4190 - accuracy: 0.8511 - val_loss: 0.4137 - val_accuracy: 0.8554
Epoch 4/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3936 - accuracy: 0.8603 - val_loss: 0.3871 - val_accuracy: 0.8598
Epoch 5/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3729 - accuracy: 0.8673 - val_loss: 0.3742 - val_accuracy: 0.8656
Epoch 6/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3557 - accuracy: 0.8723 - val_loss: 0.3875 - val_accuracy: 0.8664
Epoch 7/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3445 - accuracy: 0.8750 - val_loss: 0.3589 - val_accuracy:

### Performance Scheduling

In [46]:
model = build_model()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
             metrics=["accuracy"])

In [47]:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
history = model.fit(X_train, y_train, epochs=10,
                   validation_data=(X_valid, y_valid),
                   callbacks=[lr_scheduler])

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.6982 - accuracy: 0.7614 - val_loss: 0.4863 - val_accuracy: 0.8286 - lr: 0.0100
Epoch 2/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4716 - accuracy: 0.8333 - val_loss: 0.4449 - val_accuracy: 0.8412 - lr: 0.0100
Epoch 3/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4225 - accuracy: 0.8487 - val_loss: 0.4184 - val_accuracy: 0.8520 - lr: 0.0100
Epoch 4/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3971 - accuracy: 0.8592 - val_loss: 0.3843 - val_accuracy: 0.8654 - lr: 0.0100
Epoch 5/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3758 - accuracy: 0.8663 - val_loss: 0.3819 - val_accuracy: 0.8638 - lr: 0.0100
Epoch 6/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3587 - accuracy: 0.8712 - val_loss: 0.3846 - val_accuracy: 0.8632 - lr: 0.0100
Epoch 7/10
1719/1719 [==============================] - 4s

# Avoiding Overfitting Through Regularization

### l1 and l2 regularization

In [48]:
layer = tf.keras.layers.Dense(100, activation="relu",
                             kernel_initializer="he_normal",
                             kernel_regularizer=tf.keras.regularizers.l2(0.01))

In [49]:
tf.random.set_seed(42)

In [50]:
from functools import partial
RegularizedDense = partial(tf.keras.layers.Dense,
                          activation="relu",
                          kernel_initializer="he_normal",
                          kernel_regularizer=tf.keras.regularizers.l2(0.01))

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(100),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])

In [51]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.02)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=2,
                    validation_data=(X_valid, y_valid))

Epoch 1/2
1719/1719 [==============================] - 5s 2ms/step - loss: 3.1470 - accuracy: 0.7715 - val_loss: 1.8709 - val_accuracy: 0.8248
Epoch 2/2
1719/1719 [==============================] - 4s 2ms/step - loss: 1.4330 - accuracy: 0.8144 - val_loss: 1.1302 - val_accuracy: 0.8172


### Dropout

In [52]:
tf.random.set_seed(42)

In [53]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
                         kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(100, activation="relu",
                         kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [54]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6790 - accuracy: 0.7507 - val_loss: 0.4635 - val_accuracy: 0.8196
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5128 - accuracy: 0.8106 - val_loss: 0.4295 - val_accuracy: 0.8374
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4740 - accuracy: 0.8262 - val_loss: 0.4001 - val_accuracy: 0.8480
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4464 - accuracy: 0.8358 - val_loss: 0.3740 - val_accuracy: 0.8604
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4350 - accuracy: 0.8381 - val_loss: 0.3658 - val_accuracy: 0.8580
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4242 - accuracy: 0.8409 - val_loss: 0.3541 - val_accuracy: 0.8654
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4135 - accuracy: 0.8467 - val_loss: 0.3501 - val_accuracy:

In [55]:
model.evaluate(X_train, y_train)

1719/1719 [==============================] - 3s 2ms/step - loss: 0.3106 - accuracy: 0.8857


[0.3105733096599579, 0.8857091069221497]

In [56]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3642 - accuracy: 0.8687


[0.3641897439956665, 0.8687000274658203]

### MC Dropout

In [57]:
tf.random.set_seed(42)

In [58]:
y_probas = np.stack([model(X_test, training=True)
                    for sample in range(100)])
y_proba = y_probas.mean(axis=0)

In [59]:
model.predict(X_test[:1]).round(3)

1/1 [==============================] - 2s 2s/step


array([[0.   , 0.   , 0.   , 0.   , 0.   , 0.009, 0.   , 0.167, 0.   ,
        0.823]], dtype=float32)

In [60]:
y_proba[0].round(3)

array([0.001, 0.   , 0.001, 0.001, 0.   , 0.028, 0.   , 0.195, 0.001,
       0.773], dtype=float32)

In [61]:
y_pred = y_proba.argmax(axis=1)
accuracy = (y_pred == y_test).sum() / len(y_test)
accuracy

0.8713

### Max norm

In [63]:
dense = tf.keras.layers.Dense(100, activation="relu",
                             kernel_initializer="he_normal",
                             kernel_constraint=tf.keras.constraints.max_norm(1.))

In [65]:
MaxNormDense = partial(tf.keras.layers.Dense,
                      activation="relu", kernel_initializer="he_normal",
                      kernel_constraint=tf.keras.constraints.max_norm(1.))

tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    MaxNormDense(100),
    MaxNormDense(100),
    tf.keras.layers.Dense(10, activation="softmax")
])
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
             metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                   validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5501 - accuracy: 0.8035 - val_loss: 0.4668 - val_accuracy: 0.8210
Epoch 2/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.4110 - accuracy: 0.8511 - val_loss: 0.4091 - val_accuracy: 0.8468
Epoch 3/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3816 - accuracy: 0.8607 - val_loss: 0.3709 - val_accuracy: 0.8654
Epoch 4/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.3629 - accuracy: 0.8677 - val_loss: 0.3481 - val_accuracy: 0.8728
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3496 - accuracy: 0.8702 - val_loss: 0.3554 - val_accuracy: 0.8732
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3449 - accuracy: 0.8721 - val_loss: 0.3828 - val_accuracy: 0.8634
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3343 - accuracy: 0.8752 - val_loss: 0.4066 - val_accuracy: